![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace%20in%20Spark%20NLP%20-%20XlmRoBertaForZeroShotClassification.ipynb)

## Import XlmRoBertaForZeroShotClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- This feature is only in `Spark NLP 3.3.x` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import XLM-RoBERTa models trained/fine-tuned for token classification via `XLMRobertaForSequenceClassification` or `TFXLMRobertaForSequenceClassification`. These models are usually under `Zero-Shot Classification` category and have `xlm-roberta` in their labels
- Reference: [TFXLMRobertaForSequenceClassification](https://huggingface.co/docs/transformers/model_doc/xlm-roberta#transformers.TFXLMRobertaForSequenceClassification)
- Some [example models](https://huggingface.co/models?filter=xlm-roberta&pipeline_tag=zero-shot-classification)

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.25.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- XLMRobertaTokenizer requires the `SentencePiece` library, so we install that as well

In [1]:
!pip install -q transformers==4.25.1 tensorflow==2.11.0 sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 28.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.19.2 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which i

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [symanto/xlm-roberta-base-snli-mnli-anli-xnli](https://huggingface.co/symanto/xlm-roberta-base-snli-mnli-anli-xnli) model from HuggingFace as an example
- In addition to `TFXLMRobertaForSequenceClassification` we also need to save the `XLMRobertaTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [2]:
from transformers import TFXLMRobertaForSequenceClassification, XLMRobertaTokenizer
import tensorflow as tf

MODEL_NAME = 'symanto/xlm-roberta-base-snli-mnli-anli-xnli'

tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

# just in case if there is no TF/Keras file provided in the model
# we can just use `from_pt` and convert PyTorch to TensorFlow
try:
  print('try downloading TF weights')
  model = TFXLMRobertaForSequenceClassification.from_pretrained(MODEL_NAME)
except:
  print('try downloading PyTorch weights')
  model = TFXLMRobertaForSequenceClassification.from_pretrained(MODEL_NAME, from_pt=True)

# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask")
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

try downloading TF weights


config.json:   0%|          | 0.00/921 [00:00<?, ?B/s]

try downloading PyTorch weights


pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFXLMRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForSequenceClassification for predictions without further training.


Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {MODEL_NAME}

total 1086412
-rw-r--r-- 1 root root        921 Mar  1 23:39 config.json
drwxr-xr-x 3 root root       4096 Mar  1 23:38 saved_model
-rw-r--r-- 1 root root 1112473408 Mar  1 23:39 tf_model.h5


In [4]:
!ls -l {MODEL_NAME}/saved_model/1

total 9492
drwxr-xr-x 2 root root    4096 Mar  1 23:38 assets
-rw-r--r-- 1 root root      55 Mar  1 23:38 fingerprint.pb
-rw-r--r-- 1 root root  167607 Mar  1 23:39 keras_metadata.pb
-rw-r--r-- 1 root root 9535557 Mar  1 23:38 saved_model.pb
drwxr-xr-x 2 root root    4096 Mar  1 23:38 variables


In [5]:
!ls -l {MODEL_NAME}_tokenizer

total 4960
-rw-r--r-- 1 root root 5069051 Mar  1 23:37 sentencepiece.bpe.model
-rw-r--r-- 1 root root     280 Mar  1 23:37 special_tokens_map.json
-rw-r--r-- 1 root root     504 Mar  1 23:37 tokenizer_config.json


- as you can see, we need the SavedModel from `saved_model/1/` path
- we also be needing `sentencepiece.bpe.model` file from the tokenizer
- all we need is to copy `sentencepiece.bpe.model` file into `saved_model/1/assets` which Spark NLP will look for
- in addition to vocabs, we also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [6]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

# let's copy sentencepiece.bpe.model file to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/sentencepiece.bpe.model {asset_path}

In [7]:
# get label2id dictionary
labels = model.config.label2id
# sort the dictionary based on the id
labels = sorted(labels, key=labels.get)

with open(asset_path+'/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

In [8]:
! ls -l {asset_path}

total 4956
-rw-r--r-- 1 root root      32 Mar  1 23:39 labels.txt
-rw-r--r-- 1 root root 5069051 Mar  1 23:39 sentencepiece.bpe.model


## Import and Save XlmRoBertaForZeroShotClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [9]:
while True:
  pass
# restart here to free up ram

KeyboardInterrupt: 

In [1]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.3.0
setup Colab for PySpark 3.2.3 and Spark NLP 5.3.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.8/564.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 10.6 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [2]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `XlmRoBertaForZeroShotClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `XlmRoBertaForZeroShotClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [3]:
from sparknlp.annotator import *
from sparknlp.base import *

MODEL_NAME = 'symanto/xlm-roberta-base-snli-mnli-anli-xnli'

zero_shot_classifier = XlmRoBertaForZeroShotClassification\
  .loadSavedModel('{}/saved_model/1'.format(MODEL_NAME), spark)\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [4]:
zero_shot_classifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [5]:
! rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your XlmRoBertaForZeroShotClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [6]:
! ls -l {MODEL_NAME}_spark_nlp

total 1100412
drwxr-xr-x 4 root root       4096 Mar  1 23:44 fields
drwxr-xr-x 2 root root       4096 Mar  1 23:44 metadata
-rw-r--r-- 1 root root 1121735053 Mar  1 23:44 xlmroberta_classification_tensorflow
-rw-r--r-- 1 root root    5069051 Mar  1 23:44 xlmroberta_spp


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny XlmRoBertaForZeroShotClassification model 😊

In [7]:
zero_shot_classifier_loaded = XlmRoBertaForZeroShotClassification.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

You can see what labels were used to train this model via `getClasses` function:

In [8]:
# .getClasses was introduced in spark-nlp==3.4.0
zero_shot_classifier_loaded.getClasses()

['NEUTRAL', 'CONTRADICTION', 'ENTAILMENT']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [9]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    zero_shot_classifier_loaded
])

# couple of simple examples
example = spark.createDataFrame([["My name is Sarah and I live in London"], ['My name is Clara and I live in Berkeley, California.']]).toDF("text")

result = pipeline.fit(example).transform(example)

# result is a DataFrame
result.select("text", "class.result").show()

+--------------------+------------+
|                text|      result|
+--------------------+------------+
|My name is Sarah ...|[ENTAILMENT]|
|My name is Clara ...|   [NEUTRAL]|
+--------------------+------------+



That's it! You can now go wild and use hundreds of `XlmRoBertaForZeroShotClassification` models from HuggingFace 🤗 in Spark NLP 🚀
